In [1]:
!pip install -U sentence-transformers arxiv umap-learn nltk

Keyring is skipped due to an exception: 'keyring.backends'
Requirement already up-to-date: sentence-transformers in /opt/anaconda3/lib/python3.7/site-packages (2.2.2)
Requirement already up-to-date: arxiv in /opt/anaconda3/lib/python3.7/site-packages (1.4.7)
Requirement already up-to-date: umap-learn in /opt/anaconda3/lib/python3.7/site-packages (0.5.3)
Requirement already up-to-date: nltk in /opt/anaconda3/lib/python3.7/site-packages (3.8.1)


In [2]:
import arxiv
from sentence_transformers import SentenceTransformer
import umap
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mihailtereskin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
search = arxiv.Search(query = "quantum", max_results=350, sort_by=arxiv.SortCriterion.SubmittedDate)

In [4]:
papers = list()
for result in search.results():
    papers.append(result.title + '.' + '\n\n' + result.summary)

In [5]:
print(papers[0])

Electronic Excited States from a Variance-Based Contracted Quantum Eigensolver.

Electronic excited states of molecules are central to many physical and
chemical processes, and yet they are typically more difficult to compute than
ground states. In this paper we leverage the advantages of quantum computers to
develop an algorithm for the highly accurate calculation of excited states. We
solve a contracted Schr\"odinger equation (CSE) -- a contraction (projection)
of the Schr\"odinger equation onto the space of two electrons -- whose
solutions correspond identically to the ground and excited states of the
Schr\"odinger equation. While recent quantum algorithms for solving the CSE,
known as contracted quantum eigensolvers (CQE), have focused on ground states,
we develop a CQE based on the variance that is designed to optimize rapidly to
a ground or excited state. We apply the algorithm in a classical simulation
without noise to computing the ground and excited states of H$_{4}$ and BH.


In [6]:
#tokenize text into sentences
tokenized_papers = [nltk.sent_tokenize(paper) for paper in papers]

In [7]:
print(tokenized_papers[0])

['Electronic Excited States from a Variance-Based Contracted Quantum Eigensolver.', 'Electronic excited states of molecules are central to many physical and\nchemical processes, and yet they are typically more difficult to compute than\nground states.', 'In this paper we leverage the advantages of quantum computers to\ndevelop an algorithm for the highly accurate calculation of excited states.', 'We\nsolve a contracted Schr\\"odinger equation (CSE) -- a contraction (projection)\nof the Schr\\"odinger equation onto the space of two electrons -- whose\nsolutions correspond identically to the ground and excited states of the\nSchr\\"odinger equation.', 'While recent quantum algorithms for solving the CSE,\nknown as contracted quantum eigensolvers (CQE), have focused on ground states,\nwe develop a CQE based on the variance that is designed to optimize rapidly to\na ground or excited state.', 'We apply the algorithm in a classical simulation\nwithout noise to computing the ground and excit

In [8]:
embeddings = list()
model = SentenceTransformer('all-MiniLM-L6-v2')
for token_paper in tokenized_papers:
    embeddings.append(model.encode(token_paper))

In [26]:
import numpy as np
np.argmin([embedding.shape[0] for embedding in embeddings])
tokenized_papers[335]

['Treatment-Response Analysis of Tumor as A Quantum Particle.',
 'In this article, I present a novel and computational-efficient approach for\ntreatment-response modeling of tumor progression-free survival (PFS)\nprobability using the physical phenomenon of a quantum particle walking on a\none-dimensional lattice with the presence of a proximate trap.']

In [30]:
reducer = umap.UMAP(n_neighbors=2, init='random', n_components=2)
umap_embeddings=list(map(lambda x:reducer.fit_transform(x), embeddings))

/opt/anaconda3/lib/python3.7/site-packages/umap/umap_.py:2345: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  "n_neighbors is larger than the dataset size; truncating to "
/opt/anaconda3/lib/python3.7/site-packages/umap/umap_.py:133: UserWarning: A large number of your vertices were disconnected from the manifold.
Disconnection_distance = inf has removed 0 edges.
It has fully disconnected 2 vertices.
You might consider using find_disconnected_points() to find and remove these points from your data.
Use umap.utils.disconnected_vertices() to identify them.
  f"A large number of your vertices were disconnected from the manifold.\n"


ValueError: zero-size array to reduction operation maximum which has no identity